### Dependencies:

In [ ]:
import yaml
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [ ]:
edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")

### Functions:

In [ ]:
def resources(name:str) -> dict[str:str]:
    """Read xpath.yml file that contains xpath's to scrape"""

    with open("xpath.yml", "r") as f:
        content:dict[str:str] = yaml.safe_load(f)
        return content[name]

In [ ]:
def close_prompt(driver:object) -> None:
    """Close ad window"""

    button:str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [ ]:
def choose_category(category:str, driver:object) -> None:
    """Choose ad category"""

    driver.find_element(by=By.XPATH, value=category).click()

In [ ]:
def click_element(element:object, path:str) -> None:
    """Click element on the current document"""

    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [ ]:
def extract_description(element:object, path:str) -> str:
    """Extract ad description on the current document"""

    description:str = element.find_element(by=By.XPATH, value=path)
    return description

In [ ]:
def extract_feature(element:object, path:str) -> list[str]:
    """Extract some feature from list document based on xpath used."""

    feature = []

    try:
        feature:str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")



In [ ]:
def persistence(dataframe:pd.DataFrame) -> None:
    """Generate .csv file from data scraped"""

    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [ ]:
def parse_price(dataframe: pd.DataFrame, column:str, pattern:str) -> None:
    """
    Cast string prices to int prices.
    Works with $#,### or $#,###\n* pattern
    """


    dataframe[column] = dataframe [column] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",","")))


## Extraction 🧲

### Sedan:

Variables pool.

In [ ]:
descriptions:list[str] = []
prices:list[int] = []
models:list[str] = []
years:list[int] = []
kilometers:list[int] = []
engines:list[str] = []

# seconds
DELAY:int = 3

In [ ]:
sedan = resources("cars_sedan")

Get web & close initial prompt.

In [ ]:
edge.get(sedan["url"])

In [ ]:
# close_prompt(edge)

In [ ]:
elements = edge.find_elements(by=By.XPATH, value=sedan["ad"])

Extract descriptions.

In [ ]:
elements[0].click()
try:
    for element in range(len(elements)):
        dynamic_delay = WebDriverWait(edge, DELAY).until(EC.presence_of_element_located((By.XPATH, sedan["description"])))
        temp = extract_description(edge, sedan["description"])
        descriptions.append(temp.text)
        click_element(edge, sedan["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, sedan["exit"])

Extract prices & models.

In [ ]:
prices = extract_feature(edge, sedan["price"])
models = extract_feature(edge, sedan["model"])
models.remove("Blue Book Encuentra24")

Extract car year, engine & kilometers.

In [ ]:
years = extract_feature(edge, sedan["year"])
engines = extract_feature(edge, sedan["engine"])
kilometers = extract_feature(edge, sedan["kilometers"])

## Transformation ⚙

There exist a missing kilometer value, so in kilometers actually was append a engine type instead a kilometer value due it's value is missing.

In [ ]:
# years.remove("Ver más")
# engines.append("Gasolina")
# kilometers.remove("Gasolina")
# kilometers.insert(-2,"0")

Temporal df to apply transformation and after save it as csv.

In [ ]:
df_temp = pd.DataFrame({
    "model": models,
    "year": years,
    "kilometers": kilometers,
    "engine": engines,
    "price": prices,
    "description": descriptions
})

### Parse columns:

Chance column dtype from str -> int

In [ ]:
price_rg:str = r"^\$(?P<prices>\d+,\d+)?"
parse_price(df_temp, "price", price_rg)

In [ ]:
km_rg: str = r"(?P<km>^\d+)"
df_temp["kilometers"] = df_temp["kilometers"].str.extract(km_rg).applymap(int)

In [ ]:
df_temp["year"] = df_temp["year"].map(int)

In [209]:
persistence(df_temp)

### Stats:

Basic information about new extraction stored in df.

In [208]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   model        45 non-null     object
 1   year         45 non-null     int64 
 2   kilometers   45 non-null     int64 
 3   engine       45 non-null     object
 4   price        45 non-null     int64 
 5   description  45 non-null     object
dtypes: int64(3), object(3)
memory usage: 2.2+ KB


In [206]:
df_temp.describe()

,year,kilometers,price
count,45.000000,45.000000,45.000000
mean,2013.622222,87042.177778,8170.000000
std,5.609606,60524.943627,2799.617466
min,1992.000000,0.000000,2900.000000
25%,2011.000000,46000.000000,6500.000000
50%,2016.000000,81570.000000,8150.000000
75%,2017.000000,127580.000000,10000.000000
max,2021.000000,262634.000000,15000.000000
